# AP CSP Create Performance Task

#### Background: 
This program determines the number of inches of insulation the user should add to his or her attic in order to optimize the heating and cooling loss. The mechanism of heat loss through ceiling of a home to the attic is primarily through conduction. Conduction is when heat travels through a substance to a different temperature area in an effort to substain equillibrium. 

The optimal depth of insulation is calculated using the net savings function where the depth of insulation is the output. This application can be expanded to other states in the US or more building materials. Calculus and graphing utilities are used to determine the optimal savings. The rate of heat loss is modeled using direct and inverse variation in the net savings function. Analysis requires maximizing a rational function using either differential calculus or graphing technology. (Integration can also be brought to bear to compute the average annual temperature difference between the interior and attic.)

### Net Savings Equation: 
#### P(d) = 24 · AHDD · A · Y · CH · ( 1/R − ( 1 / (R + d · Rnper) ) ) − A · 12d · CI
* AHDD = Annual Heating Degree Day number(each city has one)
* A = the square area of the attic in square feet
* Y = the age of the home in years
* CH = efficiency of home heating in BTU's
* R = initial insulation value
* Rnper = future insulation value
* CI = cost of insulation per cubic foot

### References
* Research paper from College Mathematics Journal: https://www.jstor.org/stable/pdf/10.4169/college.math.j.43.2.165.pdf
* City AHDD values: http://www.cpc.ncep.noaa.gov/products/analysis_monitoring/cdus/degree_days/wctyhddy.txt
* Material R-values: https://www.thermaxxjackets.com/5-most-common-thermal-insulation-materials/

In [1]:
from sympy import *
from sympy import Symbol
#This function generates the net savings function based on following parameters
#Equation was refrenced from College Mathematics Journal #https://www.jstor.org/stable/pdf/
#10.4169/college.math.j.43.2.165.pdf
#AHDD = Annual Heating Degree Day number(each city has one)
#Area = the square area in square feet of the attic
#Years = the age of the home in years
#material = R-value of future insulation
#returns net savings function
def getEquation(AHDD, area, years, material):
    x = Symbol('x') #used so that the equation can interpret what "x" is
    #constants
    CH = 1/80000    #CH = efficiency of home heating in BTU's
    R = 7.1         #R = initial insulation value
    CI = 0.4        #CI = cost of insulation per cubic foot
    y = 24 * AHDD * area * years * CH * ( 1/R -(1 /(R+ x * material))) - area * CI * (x / 12) 
    return y

In [2]:
#Reference: http://stackoverflow.com/questions/9876290/how-do-i-compute-derivative-using-numpy
from sympy import Symbol
#function differentiates resulting equation from function equation()
#returns the derivative
def differentiate(formula):
    x = Symbol('x')                    #used so that the equation knows what "x" is
    formulaPrime = formula.diff(x)     #equation being differentiated
    return formulaPrime

In [3]:
#References: http://docs.sympy.org/dev/modules/solvers/solvers.html
from sympy.solvers import solve
from sympy import Symbol
#function finds the roots of the differentiated equation
#AHDD = annual heating degree day number(each city has one)
#Area = the square area in square feet of the attic
#Years = the age of the home in years
#material = R-value of future insulation
#returns the solutions of the equtaion
def findSolution(AHDD,area,years,Rvalue):
    t = Symbol('t')#used so that the function knows that it's on the domain of "t" instead of "x"
    eq = getEquation(AHDD,area,years,Rvalue)
    #solve is used to find the roots of the function
    solutions = solve(differentiate(eq))
    return solutions

In [4]:
#Reference: http://effbot.org/tkinterbook/entry.htm
#function retrieves the input from the usesr input text boxes and dropdown boxes and assigns to variables
#all of the input is gathered and returned
#returns the values from the textboxes and dropdown boxes
def getInput():
    area  = int(txtArea.get())                   #square area of attic is saved in variable area
    years = int(txtYears.get())                  #age of attic is saved in variable years.
    city  = cityDropTop.get()
    AHDD  = int(cityChoices[city])               #the value associated to the appropriate city 
                                                 # is stored in the AHDD variable
    material = materialDropTop.get()
    Rvalue = float(materialChoices[material])    #the value associated to the material chosen is stored in the variable Rvalue
    return AHDD,area,years,Rvalue

In [5]:
#function checks whether solutions are extraneous and only outputs the results that make sense
#returns the string with the positive solution in it
def showSolutions(AHDD,area,years,Rvalue):
    #prettyText is defined so that we can concatenate the message
    prettyText = "Based on the calulations of the net savings function, \
    the optimal depth of insulation recommended is "
    #for loop is used to check if the values are positive or negative. It eliminates negative ones
    for solution in findSolution(AHDD, area,years,Rvalue):
        if (solution > 0):
            prettyText += str(round(solution,2))
    prettyText += " inches"
    return prettyText

In [6]:
#Reference: https://pythonprogramming.net/how-to-embed-matplotlib-graph-tkinter-gui/
#based on user input the net saving function is generated, differentiated, and finally graphed for 
#the optimal depth displayed
#this function graphs the derivative of the net savings function in order to find the optimal 
#insulation thickness
from numpy import arange
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
def graph(eq):
    #parameter is for the equation graph() is going to plot 
    f = Figure(figsize=(6, 5), dpi=50)
    a = f.add_subplot(111)
    f.patch.set_visible(True)
    a.title.set_text('Derivative of Net Savings function')
    a.set_xlabel('Insulation Thickness(inches)')
    a.set_ylabel('a')
    #arranges the domain of the figure
    t = arange(-20, 20, 0.5)
    #eval() stores the y values of the equation in the form  
    #of a list to plot to the corresponding x-values
    s = eval(eq)
    #the graph of the parameter eq is graphed
    a.plot(t, s)
    #background canvas for the graph is created
    canvas = FigureCanvasTkAgg(f, master=root)
    #the canvas is plotted
    canvas.show()
    #canvas is formatted to be in the sixth row and column 1
    canvas.get_tk_widget().grid(row=5, column = 2, rowspan = 6)
    #the fancy buttons for graph
    toolbar_frame = Frame(root)
    #toolbar is placed in row right underneath graph 
    toolbar_frame.grid(row=15, column = 2, pady = 5)
    toolbar = NavigationToolbar2TkAgg( canvas, toolbar_frame )
    canvas._tkcanvas.grid(row=5,column=2)
    #functionality for buttons 
    canvas.mpl_connect('key_press_event', on_key_event)  

#function imports the functionality for the toolbar buttons
#home button returns the graph to its normal position after observation has been done on  it
#the left and right buttons let you go back and forth between previous views
#the axis buttons let you scroll inwhich ever drection to see more of the graph
#the rectangle button lets you zoom to a rectangle
#the the second to last button lets you change the format of the graph to your liking
#The last button lets you save the figure on your desktop
def on_key_event(event):
    key_press_handler(event, canvas, toolbar)

In [ ]:
#master function checks if all the fields are filled in. If not it shows an error message 
#displays the graph, and displays the solutions in the output text box
#function is called when user clicks "Find Insulation Thickness" button
#as long as all of the fields are filled in,application returns the inches of insulation and graph
from tkinter import messagebox
def masterFunction():
    if(len(txtArea.get()) == 0 or len(txtYears.get()) == 0):#alert pops up if any of the required fields aren't filled in
        messagebox.showinfo("Find Insulation Thickness", "Please fill in all fields.")
    else:           
        AHDD,area,years,Rvalue = getInput()      #parameter values are stored in appropriate variables
        eq = getEquation(AHDD,area,years,Rvalue) #net savings function is saved in variable eq 
        derivative = str(differentiate(eq))      #the deriative of the net savings function is saved in variable in derivative
        deq = derivative.replace("x","t")        #all x's are replaced with t's because graph() plots functions on a domain of t
        graph(deq)                               #graph of derivative is shown in tkinter window
        t = showSolutions(AHDD,area,years,Rvalue)#solutions of the derivative of the net savings function are found
        editArea.insert(tk.INSERT, t)            #solutions are displayed in solution textbox

### This is the main program that creates a graphical user interface to capture user input to calculate net savings

In [ ]:
#Reference: http://stackoverflow.com/questions/34108841/creating-a-simple-gui-program-using-tkinter-in-python
#all the necesary modules are imported from tkinter
from tkinter import Tk, Label, Entry, StringVar
from tkinter import *
import tkinter.scrolledtext as tkst
from PIL import Image, ImageTk
#because we don't know which version of tkinter works we try one and if it doesn't work we try the other
try:
    # for Python2
    import Tkinter as tk
except ImportError:
    # for Python3
    import tkinter as tk
    
#root for the tkinter window
root = Toplevel() #Reference: http://stackoverflow.com/questions/20251161/tkinter-tclerror-image-pyimage3-doesnt-exist

#title is defined
root.title("Insulation Calculator for Home Attic")

instructionText=StringVar()
instructionText.set('This program determines the number of inches \n \
                     of insulation the user should add to his or her attic \n \
                    in order to optimize the heating and cooling loss.')
                     
instructionDir=Label(root, textvariable=instructionText, height=4, font=("Helvetica 10 bold"))
instructionDir.grid(row=0,column=1,sticky=E, columnspan = 1)

image = Image.open("equation.png")
photo = ImageTk.PhotoImage(image)
label = tk.Label(root, image=photo)
label.image = photo
label.grid(row=1, column=1, sticky=E ,columnspan = 1)

image2 = Image.open("attic1.png")
photo2 = ImageTk.PhotoImage(image2)
label2 = tk.Label(root, image=photo2)
label2.image = photo2
label2.grid(row=0, column=2, rowspan = 4)

#text label for text box is defined
areaText=StringVar()
areaText.set("Area (sqft)")
areaDir=Label(root, textvariable=areaText, height=1, font=("Helvetica 10 bold"))
areaDir.grid(row=2,column=0,sticky=E, padx = 10)

#text box is created
txtArea=StringVar(None)
areaName=Entry(root,textvariable=txtArea,width=50)
areaName.grid(row=2,column=1)

#label for text box is created
yearsText=StringVar()
yearsText.set("Years")
yearsDir=Label(root, textvariable=yearsText, height=1, font=("Helvetica 10 bold"))             
yearsDir.grid(row=3,column=0,sticky=E, padx = 10)

#text box is created 
txtYears=StringVar(None)
yearsName=Entry(root,textvariable=txtYears,width=50)
yearsName.grid(row=3,column=1)

#Reference city AHDD values: #http://www.cpc.ncep.noaa.gov/products/analysis_monitoring/cdus/degree_days/wctyhddy.txt
cityText=StringVar()
cityText.set("City")
cityDir=Label(root, textvariable=cityText, height=1, font=("Helvetica 10 bold"))
cityDir.grid(row=4,column=0,sticky=E, padx = 10)
#dropdown box is created for the cities
cityDropTop = StringVar(root)
#default value is set for Chicago
cityDropTop.set('Chicago')
cityChoices = {'Chicago':4706,
           'Moline':4566,
           'Peoria':4296,
           'Quincy':4018,
           'Rockford':4874,
           'Springfield':3850, 
           'Palatine':7834}
cityOptions = OptionMenu(root, cityDropTop, *cityChoices)
cityOptions.grid(row=4,column=1,sticky=E, padx=10)

#Reference Material R-values: https://www.thermaxxjackets.com/5-most-common-thermal-insulation-materials/
materialText=StringVar()
materialText.set("Material")
materialDir=Label(root, textvariable=materialText, height=1, font=("Helvetica 10 bold"))
materialDir.grid(row=5,column=0,sticky=E, padx = 10)
#dropdown box for the insulation materials
materialDropTop = StringVar(root)
#default is set as fiberglass
materialDropTop.set('Fiberglass')
materialChoices = {'Fiberglass':3.1,
           'Mineral Wool':3.1,
           'Cellulose':3.7,
           'Polyurethane Foam':6.3,
           'Polystyrene ':4,}
materialOptions = OptionMenu(root, materialDropTop, *materialChoices)
materialOptions.grid(row=5,column=1, sticky = E, padx=10)

#button to find insulation thickness
b1 = Button(root, text='Find Optimal Insulation Thickness',command=masterFunction)
b1.grid(row=6,column=1 ,sticky=E, padx=10)

solutionText=StringVar()
solutionText.set("Solution")
solutionDir=Label(root, textvariable=solutionText, height=1, font=("Helvetica 10 bold"))
solutionDir.grid(row=10,column=0,sticky=E, padx = 10)

#textbox for solution
#Reference: http://effbot.org/tkinterbook/entry.htm
frame1 = tk.Frame(
    master = root,
)

frame1.grid(row = 10,column = 1)
editArea = tkst.ScrolledText(
    master = frame1,
    wrap   = tk.WORD,
    width  = 25,
    height = 6
)

editArea.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

#defines how large the window is when it first come out  
root.geometry("%dx%d+%d+%d" % (1000, 700, 350, 350))

root.mainloop()